Re-doing analysis to verify hypothesis presented by [Ash Hafez](https://www.kaggle.com/ashhafez/quora-question-pairs/temporal-pattern-in-train-response-rates) which suggests temporal behavior for duplicates ratio. 

I wanted to compare question posting date on quora with qid to verify the hypothesis

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

df = pd.read_csv( "../input/train.csv")

df_new      = pd.concat([df[['qid1', 'question1','is_duplicate']].rename(columns = {'qid1': 'qid','question1': 'question'}), 
                         df[['qid2', 'question2','is_duplicate']].rename(columns = {'qid2': 'qid','question2': 'question'})])
df_new      = df_new.sort_values(by=["qid"], ascending=True)
df_new["dupe_rate"] = df_new.is_duplicate.rolling(window=500, min_periods=500).mean()
df_new["timeline"]  = np.arange(df_new.shape[0]) / float(df_new.shape[0])

df_new.plot(x="timeline", y="dupe_rate", kind="line")
plt.show()

Above pattern shows temporal behaviour as reported by Ash. But, we have not removed duplicate entries for a qid, that may also have some role to play in this strong correlation.

Lets drop duplicates and check again!

In [ ]:
df_new2 = df_new.drop_duplicates(subset='qid')
df_new2["dupe_rate"] = df_new2.is_duplicate.rolling(window=500, min_periods=500).mean()
df_new2["timeline"]  = np.arange(df_new2.shape[0]) / float(df_new2.shape[0])
df_new2.plot(x="timeline", y="dupe_rate", kind="line")
plt.show()

Interesting!

So,** there is ** temporal behavior to qid but not as strong as observed in first plot. 
 
Also, in last 20% qid's it began to sharply drop. 

The sharp drop can be result of quora improving some logic to detect duplicates or just drop in quora users.

Now to verify, lets pick some significant events and try to check their correlation with qid


In [ ]:
df_new2.question.iloc[3]

I checked question with qid 3 on Quora website and found this:

Question added by Vaibhav Doshi.
What is the story of Kohinoor?
#56151487 · Thank · Report · 10 Jan 2015 10:26 AM

In [ ]:
df_new2.question.iloc[10000]

Add date on Quora for above question

Question added by Panna Pan.
How do you pronounce the name of the Danish singer "MØ" in English?
#65218948 · Thank · Report · 24 Mar 2015 1:58 PM

In [ ]:
df_new2.question.iloc[200000]

Question added by Quora User.
Atheism: Evidence the Bible is all a lie?
#14962974 · Thank · Report · 17 May 2013 9:13 PM

This qid for this is not in time order. (Did it get edited and reassigned due to other question being merged into this one )

In [ ]:
df_new2.question.iloc[300000]

Well, this question has been merged with other question

Which terror group would win if they would have fought each other Isis or Nazis? was merged into this question by Robert P. Collins.
#197856891 · Thank · Revert · Report · 23 Dec 2016 2:28 AM

In [ ]:
df_new2.question.iloc[-50000]

Actually I remember reading answers for this question last year.

Question added by Anonymous.
I am 27 years old, with a salary of 40k/month. How should I invest my money every month if my savings are 30k/month?
#144223337 · Thank · Report · 15 May 2016 7:20 PM

In [ ]:
df_new2.question.iloc[-3]

Question added by Anonymous.
Hair styling product wax, gel or clay?
#183871795 · Thank · Report · 1 Nov 2016 9:53 PM

In [ ]:
df_new2.shape

**So, indeed qids seem to be in time order.**